In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import emcee
import os

import sys
sys.path.insert(0, '../')

In [2]:
from libra import (IRTFTemplate, magnitudes, 
                   nirspec_pixel_wavelengths, throughput, kepler62, 
                   background, poisson, transit_model, transit_duration)
from copy import deepcopy
from corner import corner


sptype = 'K2V'
mag = magnitudes['Kepler-62']['J']
exptime = 100*u.s
name = 'kepler62'
planets = list('bcdef')
system = kepler62

for planet in planets:
    results_dir = os.path.join('posteriors', name)
    
    if not os.path.exists(results_dir):
        os.mkdir(results_dir)
        
    planet_params = system(planet)
    
    duration = transit_duration(planet_params)

    if np.isnan(duration):
        duration = 2/24
    
    times = np.arange(planet_params.t0 - 2*duration, 
                      planet_params.t0 + 2*duration, 
                      exptime.to(u.day).value)
    
    spectrum_photo = IRTFTemplate(sptype)
    transit = transit_model(times, planet_params)
    wl = nirspec_pixel_wavelengths()
    fluxes = np.zeros((len(times), len(wl)))

    for i in range(len(times)):
        fluxes[i, :] = poisson(spectrum_photo.n_photons(wl, exptime, mag) * transit[i] * 
                               throughput(wl) + background(wl, exptime))

    spectral_fluxes = np.sum(fluxes, axis=1)
    t, f, e = times, spectral_fluxes, np.sqrt(spectral_fluxes)

    def model(p, t, init_params):
        trial_params = deepcopy(init_params)
        trial_params.t0 = p[0]
        trial_params.rp = p[1]**0.5
        return p[2] * transit_model(t, trial_params)

    def lnlikelihood(p, t, init_params):
        return -0.5 * np.nansum((model(p, t, init_params) - f)**2 / e**2)

    def lnprior(p, t, init_params):
        t0, depth, amp = p
        if ((init_params.t0 - 0.1 < t0 < init_params.t0 + 0.1) and
            ((0.5 * init_params.rp)**2 < depth < (1.5 * init.params.rp)**2) and 
            (0.999 * np.median(f) < amp < 1.001 * np.median(f))):
            return 0.0
        return -np.inf

    ndim, nwalkers = 3, 6

    init_p = np.array([planet_params.t0, planet_params.rp**2, np.median(f)])
    pos = [init_p + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlikelihood, args=(t, planet_params))
    sampler.run_mcmc(pos, 5000);
    
    old_shape = sampler.chain.shape
    new_shape = (old_shape[0] * 2000, ndim)
    samples = sampler.chain[:, -2000:, :].reshape(new_shape)
    
    corner(samples, labels=['time', 'depth', 'f0'])
    plt.savefig('posteriors/{0}/{1}.png'.format(name, planet), bbox_inches='tight', dpi=200)
    plt.close()
    
    plt.figure()
    for i in np.random.randint(0, samples.shape[0], 100):
        step = model(samples[i, :], t, planet_params)
        plt.plot(t, step, alpha=0.05, color='k')

    #plt.errorbar(t, f, e, fmt='.', color='r')
    plt.plot(t, f, '.', color='r')
    plt.xlabel('Time [JD]')
    plt.ylabel('NIRSpec counts')
    plt.title('Assuming only Poisson errors')
    plt.savefig('posteriors/{0}/lightcurve_{1}.png'.format(name, planet), bbox_inches='tight', dpi=200)
    
    t0_mcmc, depth_mcmc, amp_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                        zip(*np.percentile(sampler.flatchain[5000:, :], [16, 50, 84],
                                                           axis=0)))
    np.savetxt('posteriors/{0}/time_solution_{1}.txt'.format(name, planet), t0_mcmc)
    plt.close()

/Users/bmmorris/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:47: RuntimeWarning: invalid value encountered in double_scalars


In [3]:
for planet in list('bcdef'):
    mid, upper, lower = np.loadtxt('posteriors/{0}/time_solution_{1}.txt'.format(name, planet), unpack=True)
    
    print("planet {0} t_rms: {1:.2f}".format(planet, ((0.5 * (upper + lower))*u.day).to(u.s)))

planet b t_rms: 13.67 s
planet c t_rms: 570.87 s
planet d t_rms: 11.90 s
planet e t_rms: 20.74 s
planet f t_rms: 28.79 s
